In [ ]:
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
import pygmt
import os

In [ ]:
key = os.environ["FIRMS_MAP_KEY"]

In [ ]:
pixels_per_tile = 512 * 2
date = "2023-06-25"
base = "https://firms.modaps.eosdis.nasa.gov/usfs/api/area/csv/"

In [ ]:
df = gpd.pd.read_csv(f"{base}{key}/VIIRS_SNPP_NRT/world/1/{date}")
df = df[df['confidence'] == 'h']
df['acq_date'] = pd.to_datetime(df['acq_date'])
df['registered'] = 1
df

In [ ]:
def rasterize_frp(df: pd.DataFrame) -> xr.Dataset:
    if len(df['acq_date'].unique()) > 1:
        raise ValueError('Multiple dates in input_dataframe')
    active = pygmt.xyz2grd(
        data=df[['longitude', 'latitude', 'registered']],
        region=[-180, 180, -90, 90],
        spacing="400e",
        duplicate="u",
        registration="p",
    ).chunk({'lon': pixels_per_tile, 'lat': pixels_per_tile})
    active = xr.where(active.notnull(), 1, 0).astype('i1')
    active = active.assign_coords({'time': df['acq_date'].iloc[0]}).expand_dims('time').astype('f2')
    active.attrs['_FillValue'] = 0
    active = active.to_dataset(name='active')
    return active

In [ ]:
burned = rasterize_frp(df)
burned

In [ ]:
store = f's3://carbonplan-scratch/firms-data/firms-{date}.zarr'
store

In [ ]:
burned.to_zarr(
    store,
    encoding={'active': {"write_empty_chunks": False, "dtype": 'i1'}},
    mode="w",
    consolidated=True,
)

## Check sparsity

In [ ]:
import zarr

In [ ]:
root = zarr.open_consolidated(store)
print(f'nchunks: {root["active"].nchunks}')
print(f'nchunks_initiliazed: {root["active"].nchunks_initialized}')
print(root["active"].dtype)

## Generate pyramids

In [ ]:
from ndpyramid import pyramid_reproject
import xarray as xr
import regionmask
import rioxarray

In [ ]:
ds = xr.open_zarr(store, consolidated=True)
print(ds.active.dtype.str)

In [ ]:
mask = regionmask.defined_regions.natural_earth_v5_0_0.countries_110.mask(ds)
mask

In [ ]:
ds = ds.where(mask == 4).fillna(0).load()
ds

In [ ]:
LEVELS = 1
dt = pyramid_reproject(ds.rio.write_crs("EPSG:4326"), levels=LEVELS, resampling="sum")

In [ ]:
for child in dt.children:
    dt[child] = xr.where(dt[child] > 0, 1, 0)
    dt[child].active.attrs['_FillValue'] = 0

In [ ]:
encoding = {
    '/0': {'burned': {"write_empty_chunks": False}},
    '/1': {'burned': {"write_empty_chunks": False}},
    '/2': {'burned': {"write_empty_chunks": False}},
    '/3': {'burned': {"write_empty_chunks": False}},
    '/4': {'burned': {"write_empty_chunks": False}},
    '/5': {'burned': {"write_empty_chunks": False}},
    # '/6': {'burned': {"write_empty_chunks": False}},
    # '/7': {'burned': {"write_empty_chunks": False}},
}

In [ ]:
output_store = f's3://carbonplan-scratch/firms-data/firms-{date}-pyramids.zarr'
dt.to_zarr(output_store, consolidated=True, encoding=encoding, mode="w")

In [ ]:
root = zarr.open_consolidated("data/FIRMS_active.zarr")
for level in ["0", "1", "2", "3", "4", "5"]:
    print(f'level {level}')
    print(f'    nchunks: {root[level]["burned"].nchunks}')
    print(f'    nchunks_initiliazed level: {root[level]["burned"].nchunks_initialized}')